In [130]:
import pandas as pd
import regex as re
import json

def extractSeqFromFileToList(file_path):
  '''This function takes a path of a fastafile and extracts all sequence names and
  sequences into a nested list [[title_0, sequence_0], [title_1, sequence_1],...]'''

  fasta_file = open(file_path, 'r')
  contents = fasta_file.readlines()
  fasta_file.close()

  fasta_list=[]

  for i in contents:
    if '>' in i:
      fasta_list.append([i.strip('>').strip(),''])
    else:
      fasta_list[-1][1] = fasta_list[-1][1]+i.strip()
  print(f"Extraction of sequence information from {file_path} finished.")
  return fasta_list


def validateDNASquence(sequence):
  '''Check if the sequence contains viable nucleotides only'''

  bases = "ATGCatgc-"
  for i in sequence:
    if i not in bases:
      print("warning, sequence doesn't contain cannonical nucleotides")
      return(False)
  return(True)


def compl(base):
  if base == "A":
    return('T')
  elif base == "T":
    return('A')
  elif base == "G":
    return('C')
  elif base == "C":
    return('G')
  elif base == "-":
    return('-')

def rev_compl(seq):
  new_seq = ""
  for base in seq:
    new_base = compl(base)
    new_seq = new_base + new_seq
  return(new_seq)

def imperfectHomologySearch(sequence, query, min_homology=0.8, fixed_errors=False):

  
  if min_homology:
    errors = round(len(query)*(1-min_homology))
  if fixed_errors:
    errors = fixed_errors
  #print(f"searching with {errors} errors...")

  output_list = re.findall( '(' + query + '){e<=' + str(errors) + '}', sequence)
  
  query_match_pairs=[rev_compl(query), output_list]
  if len(output_list) > 0:
    if len(output_list) > 1:
      print("This is unusual...")
    print(f'Found possible template(s) for {rev_compl(query)}: {output_list}')
    return(query_match_pairs)
  else:
    return([])


def findInvertedRepeat(sequence, query_length=4, min_spacer=4, imperfect_homology=False, min_homology=0.8, fixed_errors=False):

  query=sequence[:query_length]
  sequence=sequence[query_length+min_spacer:]
  query_complement = rev_compl(query)
  output_pair_list=[]

  for i in range(len(sequence)-query_length+1):
    #print(sequence[i:query_length+i])
    if imperfect_homology:
      
      output_pair=imperfectHomologySearch(sequence[i:query_length+i], query_complement, min_homology=min_homology,fixed_errors=fixed_errors)

      if output_pair != []:
        output_pair_list.append(output_pair)

    else:
      if sequence[i:query_length+i] == query_complement:
        print(f"Success, {sequence[i:query_length+i]}")
        output_pair_list.append([query, [sequence[i:query_length+i]]])

  return(output_pair_list)
    
  
def searchSequenceForRepeats(sequence, min_query_length=4, max_query_length=25, min_spacer=0, window_size=250, imperfect_homology=False, min_homology=0.8, fixed_errors=False):
  
  if imperfect_homology:
    print(f"Search has been set to find quasi-pallindromes")
    if fixed_errors:
      print(f"    Allowing up to {fixed_errors} errors/mismatches...")
    if not fixed_errors:
      print(f"    Searching with a minimum of {min_homology} homology")
  if not imperfect_homology:
    print(f"Search has been set to find perfect pallindromes")

  
  list_of_all_pairs=[]
  for query_length in range(min_query_length, max_query_length):
    print(f"###Searching for inverted-repeating {query_length}bp-long fragments...")
    sequence_size=len(sequence)
    for i in range(sequence_size-window_size+1):
      seq=sequence[i:i+window_size]
      output_pair_list = findInvertedRepeat(seq, query_length=query_length, min_spacer=min_spacer, imperfect_homology=imperfect_homology, min_homology=min_homology, fixed_errors=fixed_errors)
      for pair in output_pair_list:
        list_of_all_pairs.append(pair)

  results_dictionary={}
  print("Consolodating Results...")
  for pair in list_of_all_pairs:
    query=pair[0]
    if query in results_dictionary:
      if pair[1][0] not in results_dictionary[query]:
        results_dictionary[query].append(pair[1][0])
    else:
      results_dictionary[query] = [pair[1][0]]
  print("Results were consolidated...")
  return(results_dictionary)



### Upload a sequence

In [131]:
#Directly set the string
sequence='ATGTCCACAAAATCATATACCAGTAGAGCTGAGACTCATGCAAGTCCGGTTGCATCGAAACTTTTACGTTTAATGGATGAAAAGAAGACCAATTTGTGTGCTTCTCTTGACGTTCGTTCGACTGATGAGCTATTGAAACTTGTTGAAACGTTGGGTCCATACATTTGCCTTTTGAAAACACACGTTGATATCTTGGATGATTTCAGTTATGAGGGTACTGTCGTTCCATTGAAAGCATTGGCAGAGAAATACAAGTTCTTGATATTTGAGGACAGAAAATTCGCCGATATCGGTAACACAGTCAAATTACAATATACATCGGGCGTTTACCGTATCGCAGAATGGTCTGATATCACCAACGCCCACGGGGTTACTGGTGCTGGTATTGTTGCTGGCTTGAAACAAGGTGCGCAAGAGGTCACCAAAGAACCAAGGGGATTATTGATGCTTGCTGAATTGTCTTCCAAGGGTTCTCTAGCACACGGTGAATATACTAAGGGTACCGTTGATATTGCAAAGAGTGATAAAGATTTCGTTATTGGGTTCATTGCTCAGAACGATATGGGAGGAAGAGAAGAAGGGTTTGATTGGCTAATCATGACCCCAGGTGTAGGTTTAGACGACAAAGGCGATGCATTGGGTCAGCAGTACAGAACCGTCGACGAAGTTGTAAGTGGTGGATCAGATATCATCATTGTTGGCAGAGGACTTTTCGCCAAGGGTAGAGATCCTAAGGTTGAAGGTGAAAGATACAGAAATGCTGGATGGGAAGCGTACCAAAAGAGAATCAGCGCTCCCCATTAA'

#Or set file path
#fasta_list=extractSeqFromFileToList("/content/Lys2.fa")
#sequence = fasta_list[0][1]

#validate sequence

print(f"The uploaded sequence has {len(sequence)}bp")
if validateDNASquence(sequence):
  print("### Sequence validated ###")

The uploaded sequence has 804bp
### Sequence validated ###


In [ ]:
#Set repeat Search Parameters


min_query_length=5        # Sets min length of a query sequence
max_query_length=30       # Sets max length of a query sequence
min_spacer=0              # Set min distance between query and the repeat
window_size=700           # Sets window size within which the search is confined
imperfect_homology=True   # Set True/False, to search for imperfect/perfect homologies.
min_homology=0.9          # Sets minimum homology treshold (a fraction) when imperfect_homology=True,  
fixed_errors=1            # Sets maximum number of errors (del/sub) when imperfect_homology=True (set to False or to an integer)

#Run the search
results_dictionary = searchSequenceForRepeats(sequence=sequence,
                         min_query_length=min_query_length,
                         max_query_length=max_query_length,
                         min_spacer=min_spacer,
                         window_size=window_size,
                         imperfect_homology=imperfect_homology,
                         min_homology=min_homology,
                         fixed_errors=fixed_errors
                         )

In [ ]:
#save output as json file (and print it out to console)

output_file_name='json_output.json'

with open(output_file_name, 'w') as outfile:
    json.dump(results_dictionary, outfile)
    

json_object = json.dumps(results_dictionary, indent = 6) 
print(json_object)